In [3]:
import os
import pickle
import numpy as np
import pandas as pd
import cv2 as cv
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm_notebook
from scipy.spatial.distance import hamming, cosine

%matplotlib inline

# Dataset utils and preprocessing

In [ ]:
def image_loader(path, size):
    # String path to image
    # Tuple size of output image
    image = cv.imread(path)
    image = cv.cvtColor(image, cv.COLOR_BAYER_BGR2RGB)
    image = cv.resize(image, size, cv.INTER_CUBIC)

    return image